# AI기법과 활용 - Week 07
Google API를 활용
____

In [ ]:
# 지난시간에 세팅한 인증파일 세팅
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/jungwons/Desktop/Lecture/2022123124/practice/week-06/glassy-life-350815-07dd97f00324.json"


## 1. API를 사용하기 위한 Python Client 설치

In [ ]:

import sys
!{sys.executable} -m pip install --upgrade google-cloud-vision google-cloud-texttospeech google-cloud-speech
!pip install --upgrade google-cloud-vision google-cloud-texttospeech google-cloud-speech


## 2. Image-to-Text

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')
    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))
    return texts[0].description

In [ ]:
from IPython.display import Image
Image(filename="image-only-korean.jpeg") 


In [ ]:
text = detect_text("image-only-korean.jpeg")

In [ ]:
text

## 3. Text-to-Speech

In [ ]:
"""Synthesizes speech from the input string of text or ssml.
Make sure to be working in a virtual environment.

Note: ssml must be well-formed according to:
    https://www.w3.org/TR/speech-synthesis/
"""
from google.cloud import texttospeech
def gen_audio(text):
    # Instantiates a client
    client = texttospeech.TextToSpeechClient()

    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request, select the language code ("en-US") and the ssml
    # voice gender ("neutral")
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16
    )

    # Perform the text-to-speech request on the text input with the selected
    # voice parameters and audio file type
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # The response's audio_content is binary.
    with open("output.wav", "wb") as out:
        # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "output.wav"')

In [ ]:
gen_audio("안녕하세요")

In [ ]:
import IPython
IPython.display.Audio("output.wav")


In [ ]:
gen_audio(text)

In [ ]:
IPython.display.Audio("output.wav")


## 4. Speech-to-Text

In [ ]:
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    import io

    client = speech.SpeechClient()

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=24000,
        language_code="ko-KR",
    )

    response = client.recognize(config=config, audio=audio)
    print(response)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u"Transcript: {}".format(result.alternatives[0].transcript))
    return response.results[0].alternatives[0].transcript

In [ ]:
text = transcribe_file("output.wav")

In [ ]:
text

In [ ]:

Image(filename="image-only-korean.jpeg") 
